In [10]:
from IPython.display import clear_output,display_svg,HTML,display
from ipywidgets.widgets import IntSlider,FloatSlider,interact
import numpy as np
from random import randint,random,choices
from graphviz import Digraph
from pprint import pprint
from ordered_set import OrderedSet
from math import ceil
from itertools import permutations
from matplotlib import pyplot as mpl

def full_graph(cities,dist_table):
    graph = Digraph(engine='circo')
    for i in range(0,cities):
        graph.node(str(i),shape='point')
    for edge in dist_table:
        t = list(edge)
        graph.edge(str(t[0]),str(t[1]),dir='none',color='gray85',)
    return graph

def path_graph(cities,dist_table,path):
    graph = Digraph(engine='circo')
    for i in range(0,cities):
        graph.node(str(i))
    for edge in dist_table:
        t = list(edge)
        if edge in path:
            graph.edge(str(t[0]),str(t[1]),dir='none',color='red',label=str(dist_table[edge][0]),labelfontsize='15')
        else:
            graph.edge(str(t[0]),str(t[1]),dir='none',color='gray85')
    return graph

def get_random_set(count,size):
    a = [i for i in range(size)]
    permutation = list(permutations(a,count))
    return permutation[randint(0,len(permutation)-1)]

def generate_cities(count_of_cities,max_dist):
    path_map = {}
    for i in range(count_of_cities-1):
        for j in range(i,count_of_cities):
            if not i == j: 
                path_map[frozenset([i,j])] = randint(1,max_dist)
    return path_map

In [11]:
class Creature:
    def __init__(self,features):
        self.features = OrderedSet(features)

    def health(self,path_map):
        path_length = 0
        path_length += path_map[frozenset([self.features[0],self.features[-1]])]
        for i in range(len(self.features)-1):
            path_length += path_map[frozenset([self.features[i],self.features[i+1]])]
        return 1/path_length

    def mutate(self,mutation_p):
        if random()<mutation_p:
            pos = get_random_set(2,len(self.features))
            temp_list = list(self.features)
            temp_list[pos[0]],temp_list[pos[1]] = temp_list[pos[1]],temp_list[pos[0]]
            self.feature = OrderedSet(temp_list)

    def crossover(self,another_creature,crossover_k):
        set_a = self.features
        set_b = another_creature.features

        pos_a = get_random_set(
            ceil(
                len(set_a)*(crossover_k)
            ),
            len(set_a)
        )
        
        pos_b = []
        for pos in pos_a:
            pos_b.append(set_b.index(set_a[pos]))
        pos_b.sort()

        set_result = OrderedSet()
        for i,feature_b in enumerate(set_b):
            if i in pos_b:
                set_result.add(
                    set_a[pos_a[pos_b.index(i)]]
                )
            else:
                set_result.add(
                    feature_b
                )
        return Creature(set_result)

    def __str__(self):
        return str(list(self.features))

    def __repr__(self):
        return str(list(self.features))

class Population:
    def __init__(self,parent_count = 10,city_count=5,max_dist=10,mutation_p=0.2,crossover_k=0.5):
        self.path_map = generate_cities(city_count,max_dist)
        self.parent_population = [Creature(get_random_set(city_count,city_count)) for i in range(parent_count)]
        self.population = []
        self.best_solutions = []
        self.healths = []
        self.avr_health = []
        self.max_health = []
        self.city_count = city_count
        self.max_dist = max_dist
        self.mutation_p = mutation_p
        self.crossover_k = crossover_k
        self.parent_count = parent_count
    def step(self):
        '''
            начальная популяция - родительская, скрещиваем получаем новую, мутируем, проверяем здоровье, делаем новую родительскую
        '''
        self.recombination()
        self.health_check()
        self.remove_duplicate_from_population()
        self.selection()

    def best_solution(self):
        return max(self.best_solutions,key=lambda a: a.health(self.path_map))

    def recombination(self):
        self.population = []
        for i in self.parent_population:
            for j in self.parent_population:
                if i!=j:
                    new_creature = i.crossover(j,self.crossover_k)
                    new_creature.mutate(self.mutation_p)
                    self.population.append(new_creature)

    def health_check(self):
        self.healths = []
        self.healths = list(map(lambda a: a.health(self.path_map),self.population))
        self.avr_health.append(
            sum(self.healths)/len(self.healths)
        )
        self.max_health.append(
            max(self.healths)
        )
        self.best_solutions.append(
            self.population[
                self.healths.index(max(self.healths))
            ]
        )

    def remove_duplicate_from_population(self):
        t_dict = {}
        for creature in self.population:
            t_dict[tuple(creature.features)] = creature
        self.population = list(t_dict.values())

    def selection(self):
        #self.parent_population = sorted(self.population,key=lambda a: a.health(self.path_map))[:self.parent_count]
        self.parent_population = choices(
            self.population,
            list(
                map(
                    lambda a:a.health(self.path_map),
                    self.population
                )
            )
            ,k=self.parent_count
        )
    def run(self,steps):
        for step in range(steps):
            self.step()
        mpl.plot(self.avr_health)
        mpl.show()
        mpl.plot(self.max_health)
        mpl.show()
        print(self.best_solution())

In [12]:
@interact(
    steps = IntSlider(min=2,max=1000),
    start_population = IntSlider(min=5,max=100),
    city_count = IntSlider(min=4,max=25),
    max_dist = IntSlider(min=2,max=100),
    mutation_p = FloatSlider(min = 0.1, max=1.0),
    crossover_k = FloatSlider(min = 0.1, max=1.0)
)
def ants_alg(steps,start_population,city_count,max_dist,mutation_p,crossover_k):
    Population(start_population,city_count,max_dist,mutation_p,crossover_k).run(steps)

interactive(children=(IntSlider(value=2, description='steps', max=1000, min=2), IntSlider(value=5, description…